# Ingestion of metadata

## Loading dataset

In [1]:
import pandas as pd

In [2]:
data = pd.read_json("../../data/raw/binge_apporved_animation.json", lines=True)
data

available  \
0     [{'availability_type': 'subscription', 'channe...   
1     [{'availability_type': 'subscription', 'channe...   
2     [{'channel': 'Apple TV+', 'channel_code': 'APP...   
3     [{'availability_type': 'purchase', 'channel': ...   
4     [{'channel': 'NLZIET', 'channel_code': 'NLZT',...   
...                                                 ...   
2371  [{'availability_type': 'subscription', 'channe...   
2372  [{'availability_type': 'rental', 'channel': 'A...   
2373  [{'channel': 'Disney+', 'channel_code': 'DINY'...   
2374  [{'channel': 'Netflix', 'channel_code': 'NETX'...   
2375  [{'availability_type': 'purchase', 'channel': ...   

                                            bindinc_api  \
0     [{'rel': 'self', 'schema_level': 'program', 'u...   
1     [{'rel': 'self', 'schema_level': 'program', 'u...   
2     [{'rel': 'self', 'schema_level': 'program', 'u...   
3     [{'rel': 'self', 'schema_level': 'program', 'u...   
4     [{'rel': 'self', 'schema_level': 'program', 'u...   
...                                                 ...   
2371  [{'rel': 'self', 'schema_level': 'program', 'u...   
2372  [{'rel': 'self', 'schema_level': 'program', 'u...   
2373  [{'rel': 'self', 'schema_level': 'program', 'u...   
2374  [{'rel': 'self', 'schema_level': 'program', 'u...   
2375  [{'rel': 'self', 'schema_level': 'program', 'u...   

                                          cast_and_crew  \
0     [{'character': 'Stumpy Woodcutter', 'credit_na...   
1     [{'character': 'Mr. Primm', 'credit_name': 'Ma...   
2     [{'character': 'Elwood', 'credit_name': 'Rory ...   
3     [{'character': 'Glinda', 'credit_name': 'Laure...   
4     [{'character': 'Pitch 5 Man', 'credit_name': '...   
...                                                 ...   
2371  [{'character': 'Grug (voice)', 'credit_name': ...   
2372  [{'credit_name': 'Nicky Phelan', 'credit_role'...   
2373  [{'credit_name': 'Michael Black', 'credit_role...   
2374  [{'credit_name': 'Kazuma Shimizu', 'credit_rol...   
2375  [{'credit_name': 'Oleh Malamuzh', 'credit_role...   

                                   content_descriptions  \
0     [{'category': 'short', 'language': 'nl', 'leve...   
1     [{'category': 'short', 'language': 'nl', 'leve...   
2     [{'category': 'long', 'language': 'nl', 'level...   
3     [{'category': 'short', 'language': 'nl', 'leve...   
4     [{'category': 'short', 'language': 'nl', 'leve...   
...                                                 ...   
2371                                                NaN   
2372                                                NaN   
2373  [{'category': 'long', 'language': 'nl', 'level...   
2374  [{'category': 'short', 'language': 'nl', 'leve...   
2375  [{'category': 'long', 'language': 'nl', 'level...   

                                            identifiers  \
0     [{'id': 'tt5198068', 'level': 'program', 'sour...   
1     [{'id': 'tt11003866', 'level': 'program', 'sou...   
2     [{'id': 'tt12820696', 'level': 'program', 'sou...   
3     [{'id': 'tt4429160', 'level': 'program', 'sour...   
4     [{'id': 'tt6324278', 'level': 'program', 'sour...   
...                                                 ...   
2371  [{'id': 'tt22458454', 'level': 'program', 'sou...   
2372  [{'id': 'tt21880400', 'level': 'program', 'sou...   
2373  [{'id': 'tt28477869', 'level': 'program', 'sou...   
2374  [{'id': 'tt15120908', 'level': 'program', 'sou...   
2375  [{'id': 'tt6685538', 'level': 'program', 'sour...   

                                             production schema_level  \
0     {'country': ['ie', 'lu', 'us', 'gb', 'fr'], 'o...      program   
1     {'country': ['us'], 'original_length': 1500, '...      program   
2     {'country': ['us'], 'original_length': 1500, '...      program   
3     {'country': ['kr'], 'original_length': 5520, '...      program   
4     {'country': ['us', 'cn', 'jp'], 'original_leng...      program   
...                                                 ...          ... 

## Converting structured data into natural language

The following functions makes a compilation of some of the metadata above, turning the information into a fluid human-readable text. This is what the Large Language Model will use for reasoning, which is compatible to the inputs (questions) provided by the users.

In [3]:
def desc_person(details):
    """receives a dict"""
    
    if pd.isna(details):
        return ""
    
    text = details['credit_name']
    text = f"{text} is the {details['credit_role']}"
    if "character" in details:
        text = f"{text} that plays {details['character']}"
    return f"{text}.".replace("\n"," ")



def desc_cast_crew(details):
    """receives an array of dicts"""
    if type(details) != list:
        return ""
    
    return "; ".join([desc_person(d) for d in details if d["credit_role"] in ["actor","director","composer"]])



def desc_avail(details, title="It"):
    """receives an array of dicts"""
    
    if type(details) != list:
        return ""
    
    rent = [av for av in details if "availability_type" in av and av["availability_type"]=="rental"]
    purch = [av for av in details if "availability_type" in av and av["availability_type"]=="purchase"]
    subs = [av for av in details if "availability_type" in av and av["availability_type"]=="subscription"]
    unk = [av for av in details if "availability_type" not in av]
    
    text = ""
    if len(rent) > 0:
        text = f"{text}{title} is available for rental on {', '.join(set([av['channel'] for av in rent]))}."
        
    if len(purch) > 0:
        text = f"{text}{title} is available for purchase on {', '.join(set([av['channel'] for av in purch]))}."
        
    if len(subs) > 0:
        text = f"{text}{title} is available for subscription on {', '.join(set([av['channel'] for av in subs]))}."
        
    if len(unk) > 0:
        text = f"{text}{title} is available on {', '.join(set([av['channel'] for av in unk]))}."
    return text



def desc_title(details):
    """receives a dict"""
    
    if pd.isna(details):
        return ""
    
    if "title" not in details:
        return list(details.keys())[0]
        
    text = details["title"]
    
    others = []
    if "original_title" in details:
        others.append(f"original title: {details['original_title']}")
    if "alternative_title" in details:
        others.append(f"alt title: {details['alternative_title']}")
        
    if len(others) > 0:
        text = f"{text} ({'; '.join(others)})"
        
    return f"{text}"



def desc_category(details, title):
    """receives a dict"""
    
    if pd.isna(details):
        return ""
    
    text = f"{title} is a {details['category']} ({', '.join(details['genres'])})"
    return f"{text}. "



import numpy as np
import subprocess
import time

def translate_to_english(txt):
    # random delay not to upset the server
    time.sleep(np.random.random()*5)
    
    txt = txt.replace("'","''").replace("\n","")
    command = f"curl --header 'Accept: text/json' --user-agent 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.8; rv:21.0) Gecko/20100101 Firefox/21.0' --data-urlencode 'client=gtx' --data-urlencode 'sl=auto' --data-urlencode 'tl=en' --data-urlencode 'dt=at' --data-urlencode 'q={txt}' -sL http://translate.googleapis.com/translate_a/single | jq -r '.[5][][2][0][0]'"
#     print(command)
    out = subprocess.run(command, shell=True, stdout=subprocess.PIPE)
    
    return out.stdout.decode("utf-8").replace("\n","")



def desc_desc(details, translate=False):
    """receives an array of dicts"""
    
    if type(details) != list:
        return ""
    
    txt = "".join([desc["text"] for desc in details])
    if translate:
        txt = translate_to_english(txt)
        
    return f"Synopsis: {txt}"

So a current metadata

In [4]:
data.loc[42]["available"]

[{'availability_type': 'subscription',
  'channel': 'Amazon Prime Video',
  'channel_code': 'AMPV',
  'deeplink': 'https://www.primevideo.com/detail/B08KFJNNDT',
  'from': '2020-10-09 22:00:00',
  'identifier': 146739,
  'package_info': 'mojinl',
  'quality': 'hd',
  'to': '2022-02-13 05:31:54'},
 {'channel': 'Netflix',
  'channel_code': 'NETX',
  'deeplink': 'https://www.netflix.com/nl/title/70281421',
  'from': '2015-04-14 00:00:00',
  'identifier': 21553,
  'to': '2019-09-10 00:00:00'},
 {'availability_type': 'subscription',
  'channel': 'Amazon Prime Video',
  'channel_code': 'AMPV',
  'deeplink': 'https://www.primevideo.com/detail/B08KFJNNDT',
  'from': '2020-10-09 22:00:00',
  'identifier': 410792,
  'package_info': 'MojiTV',
  'quality': 'hd',
  'to': '3333-01-01 00:00:00'}]

becomes fluid text:

In [5]:
desc_avail(data.loc[42]["available"])

'It is available for subscription on Amazon Prime Video.It is available on Netflix.'

## Batch processing

Applying this conversion over the whole dataset so everything is ready to be ingested. Each title is going to be considered as a chunk of data.

In [6]:
from tqdm import tqdm
tqdm.pandas()

def row2chunk(x):
    return (
        desc_category(x["typology"], desc_title(x["title"]))
        +"\n"+
        desc_desc(x["content_descriptions"],translate=False)
        +"\n"+
        desc_cast_crew(x["cast_and_crew"])
        +"\n"+
        desc_avail(x["available"], desc_title(x["title"]))
    )

chunks = data.progress_apply(
    lambda x: {
        "page_content": row2chunk(x),
        "metadata": {
            "id": x.name,
            "bindinc_uri": x["bindinc_api"][0]["uri"]
        }
    }
    ,axis=1
)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2376/2376 [00:00<00:00, 24392.53it/s]


Each chunk of data is going to be ingested as described below. In addition to the text chunk itself, some references are appended as metadata in order to retrieve specific information from the dataset to feed some application interface features.

In [7]:
chunks[42]

{'page_content': "Winx Club: Het Magische Avontuur is a Film (Animatie). \nSynopsis: Het nieuwe leven voor Bloom in haar nieuwe koninkrijk is begonnen en ze is erg gelukkig met haar geliefde Sky. Het is echter niet allemaal rozengeur en maneschijn: het leven in Domino heeft ook zo zijn verplichtingen.\nIginio Straffi is the director.; Domitilla D'Amico is the actor that plays Tecna / Livi / Piff (voice).; Monica Migliori is the actor that plays Belladonna (voice).; Davide Perino is the actor that plays Timmy (voice).; Valeria Vidali is the actor that plays Stormy (voice).; Pasquale Anselmo is the actor that plays Liliss (voice).; Gemma Donati is the actor that plays Musa / Digit (voice).; Laura Lenghi is the actor that plays Aisha (voice).; Perla Liberatori is the actor that plays Stella / Chatta (voice).; Francesco Prando is the actor that plays Oritel (voice).; Claudia Catani is the actor that plays Giornalista (voice).; Emanuela Rossi is the actor that plays Faragonda (voice).; Emil

In [8]:
# print(chunks[42]["page_content"])

## Ingestion

From `chunks` array (of snippets/text chunks describing titles), we can now enclose them in `Document` objects that will be indexed in a vector database.

In [1]:
# TASK: Import the libraries for the FAISS vector store and the embeddings from VertexAI
# TASK: Import the Document object to be indexed

In [2]:
# TASK: Create an array of Document objects from chunks array

In [11]:
# TASK: Define the VertexAI Embeddings from one of the following deployments:
# - textembedding-gecko@001 (stable)
# - textembedding-gecko@latest (latest one, not stable) 
# - textembedding-gecko-multilingual@latest (ideal, since we have dutch text in data)

In [12]:
# TASK: create a vector database using the array of Document objects and the VertexAI Embeddings

In [3]:
# TASK: Save the vector database to build an API